In [1]:
import cv2
import pandas as pd

from ultralytics import YOLO

import zmq
from time import time

In [2]:
def check_delay(
        port,
    ):
    model = YOLO("runs/detect/640/yolov8n-d1-e100-at-2203240811/weights/best.pt")
    model.to("cpu")
    context = zmq.Context()
    zmq_socket = context.socket(zmq.PULL)
    zmq_socket.connect(f"tcp://127.0.0.1:{port}")
    count = 0

    result = []
    while True:
        ts = time()
        msg: zmq.Message = zmq_socket.recv_pyobj()
        time_before_model = time()
        frame = model(msg["frame"])
        time_model = time() - time_before_model
        result.append(
            msg | {
                "time_begin_check_delay": ts,
                "time_before_model": time_before_model,
                "time_job_model": time_model 
            }
        )
        count += 1
        if (cv2.waitKey(1) & 0xFF == ord('q')): break
        
        if (count % 100) == 0:
            result = pd.DataFrame(result)
            result.to_csv(f"{port}.csv")
            break

In [3]:
import multiprocessing as mp

num_proc = 2

with mp.Pool(num_proc) as p:
    p.map(check_delay, [port for port in range(5550, 5550 + num_proc)])


0: 640x640 (no detections), 505.0ms
Speed: 85.3ms preprocess, 505.0ms inference, 34.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 71.2ms
Speed: 1.3ms preprocess, 71.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 70.5ms
Speed: 1.4ms preprocess, 70.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 72.8ms
Speed: 1.2ms preprocess, 72.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 72.6ms
Speed: 1.9ms preprocess, 72.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 75.8ms
Speed: 1.6ms preprocess, 75.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 69.1ms
Speed: 2.0ms preprocess, 69.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 68.1ms
Speed: 1.4ms preprocess, 68.1